In [1]:
import scanpy as sc
import pandas as pd
import os
from dca.api import dca
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

2021-08-19 10:36:13.730703: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-19 10:36:13.730721: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/kopt/config.py:60: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _config = yaml.load(open(_config_path))


### INPUT: Specify which sample you want to analyze. Currently available are:

- Sample2_70_percent
- Sample2_80_percent
- Sample3_default
- Sample4_70_percent
- Sample4_80_percent
- Sample5_70_percent
- Sample5_80_percent

In [2]:
sample = 'Sample5_80_percent'

### Loading the read count data and potentially removing cell doublets/mulitplets.

In [3]:
dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/tsv/' + sample + '.barcode.cell.distribution.tsv',sep='\t')
remove_mixed = False
if remove_mixed:
    clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/'+ sample + '/tsv/cluster_assignment.tsv',
                            sep='\t')
    drop_rows = clust_file['Barcode'][clust_file['CellType']=='Mixed']
    dat = dat.drop(drop_rows)

### Creating an AnnData object from the read counts.

In [4]:
adata = sc.AnnData(dat)
sc.pp.filter_genes(adata,min_cells=1)

### Running the autoencoder: Currently, we have the following available architectures:

- 'meth-encoder': A classical autoencoder, where the dispersion parameters of both NB-distributions can be freely selected by the model.
- 'meth-encoder-constant': An autoencoder, where the dispersion parameters is fixed for each gene. This lowers the number of paramters substantially.
- 'meth-encoder-poisson': An autoencoder, which mixes a negative bionomial distribution (foreground) with a Poisson distribution as the background
- 'meth-encoder-poisson-constant': Same as the above, but with a fixed dispersion for each gene
- 'meth-encoder-simple': Instead of having a dense layer representing the mean per cell and amplicon, we use a constant mean per amplicon for both the foreground and the background distribution.

In [5]:
res = dca(adata,
          ae_type='meth-encoder',
          return_info=True,
          return_model=True,
          return_bottleneck=True,
          verbose=True,
          early_stop=50,
          epochs=300,
          network_kwds={'debug': True},
          init='glorot_normal',
          hidden_size=(16,8,16))

2021-08-19 10:36:15.619300: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-19 10:36:15.619445: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-19 10:36:15.619457: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-19 10:36:15.619474: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CZC041BTPK): /proc/driver/nvidia/version does not exist


dca: Successfully preprocessed 199 genes and 5517 cells.



2021-08-19 10:36:15,840 [WARNING] From /users/lvelten/project/Methylome/src/error_mod/dca/dca/train.py:41: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

2021-08-19 10:36:15.842868: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 199)]        0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 16)           3200        count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16)           48          enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 16)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

2021-08-19 10:36:16.764714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-08-19 10:36:16.789078: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


...

/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


4965/4965 [==============================] - 0s 70us/sample - loss: 100.1812 - val_loss: 41.9903
Epoch 3/300
4965/4965 [==============================] - 0s 74us/sample - loss: 17.1962 - val_loss: 9.8481
Epoch 4/300
4965/4965 [==============================] - 0s 55us/sample - loss: 8.9905 - val_loss: 8.2929
Epoch 5/300
4965/4965 [==============================] - 0s 66us/sample - loss: 7.8323 - val_loss: 7.3660
Epoch 6/300
4965/4965 [==============================] - 0s 55us/sample - loss: 7.1487 - val_loss: 6.9820
Epoch 7/300
4965/4965 [==============================] - 0s 54us/sample - loss: 6.9271 - val_loss: 6.8784
Epoch 8/300
4965/4965 [==============================] - 0s 61us/sample - loss: 6.8772 - val_loss: 6.8588
Epoch 9/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.8655 - val_loss: 6.8488
Epoch 10/300
4965/4965 [==============================] - 0s 57us/sample - loss: 6.8571 - val_loss: 6.8401
Epoch 11/300
4965/4965 [=============================

4965/4965 [==============================] - 0s 55us/sample - loss: 6.7953 - val_loss: 6.7858
Epoch 79/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.7956 - val_loss: 6.7853
Epoch 80/300
4965/4965 [==============================] - 0s 57us/sample - loss: 6.7950 - val_loss: 6.7850
Epoch 81/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.7951 - val_loss: 6.7847
Epoch 82/300
4965/4965 [==============================] - 0s 63us/sample - loss: 6.7963 - val_loss: 6.7844
Epoch 83/300
4965/4965 [==============================] - 0s 61us/sample - loss: 6.7950 - val_loss: 6.7848
Epoch 84/300
4965/4965 [==============================] - 0s 53us/sample - loss: 6.7941 - val_loss: 6.7832
Epoch 85/300
4965/4965 [==============================] - 0s 55us/sample - loss: 6.7943 - val_loss: 6.7847
Epoch 86/300
4965/4965 [==============================] - 0s 64us/sample - loss: 6.7943 - val_loss: 6.7837
Epoch 87/300
4965/4965 [==========================

Epoch 153/300
4965/4965 [==============================] - 0s 54us/sample - loss: 6.7807 - val_loss: 6.7719
Epoch 154/300
4965/4965 [==============================] - 0s 57us/sample - loss: 6.7823 - val_loss: 6.7720
Epoch 155/300
4965/4965 [==============================] - 0s 70us/sample - loss: 6.7827 - val_loss: 6.7720
Epoch 156/300
4965/4965 [==============================] - 0s 66us/sample - loss: 6.7809 - val_loss: 6.7719
Epoch 157/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.7812 - val_loss: 6.7719
Epoch 158/300
4965/4965 [==============================] - 0s 54us/sample - loss: 6.7804 - val_loss: 6.7719
Epoch 159/300
4965/4965 [==============================] - 0s 64us/sample - loss: 6.7814 - val_loss: 6.7718
Epoch 160/300
4965/4965 [==============================] - 0s 63us/sample - loss: 6.7812 - val_loss: 6.7719
Epoch 161/300
4965/4965 [==============================] - 0s 60us/sample - loss: 6.7824 - val_loss: 6.7719
Epoch 162/300
4965/4965 [===

Epoch 221/300
4965/4965 [==============================] - 0s 57us/sample - loss: 6.7809 - val_loss: 6.7718
Epoch 222/300
4965/4965 [==============================] - 0s 61us/sample - loss: 6.7826 - val_loss: 6.7717
Epoch 223/300
4965/4965 [==============================] - 0s 54us/sample - loss: 6.7806 - val_loss: 6.7717
Epoch 224/300
4965/4965 [==============================] - 0s 58us/sample - loss: 6.7801 - val_loss: 6.7717
Epoch 225/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.7815 - val_loss: 6.7717
Epoch 226/300
4965/4965 [==============================] - 0s 58us/sample - loss: 6.7806 - val_loss: 6.7717
Epoch 227/300
4965/4965 [==============================] - 0s 58us/sample - loss: 6.7807 - val_loss: 6.7717
Epoch 228/300
4965/4965 [==============================] - 0s 57us/sample - loss: 6.7821 - val_loss: 6.7717
Epoch 229/300
4965/4965 [==============================] - 0s 56us/sample - loss: 6.7814 - val_loss: 6.7718
Epoch 230/300
4832/4965 [===

### Writing the data on disk.

In [6]:
di = '/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/methylation_autoencoder/'
if not os.path.isdir(di):
    os.mkdir(di)
    
pd.DataFrame(adata.obsm['X_meth_value']).to_csv(di + 'mixture_prob_dca_test.csv')

In [7]:
foo = res.get_encoder().predict([adata.X, adata.X.mean(1)])

In [8]:
pd.DataFrame(adata.obsm['X_bottleneck']).to_csv(di + 'bottleneck.csv')

In [9]:
adata.obsm['mean1_norm']

array([[ 3201.415 , 33807.57  , 47586.242 , ..., 16341.072 , 12737.987 ,
         7052.2705],
       [ 6809.6797, 34775.082 , 23742.154 , ...,  8528.204 ,  4665.371 ,
        10822.638 ],
       [ 3797.4695, 37645.824 , 36656.6   , ..., 12467.072 , 12121.334 ,
        17415.514 ],
       ...,
       [11586.969 , 45598.39  , 36234.965 , ..., 14381.625 , 20681.67  ,
        22036.402 ],
       [ 5797.2627, 27189.355 , 20298.021 , ...,  7200.931 ,  3962.354 ,
         8660.505 ],
       [ 6482.4053, 39885.234 , 25736.547 , ...,  6097.3853,  6350.377 ,
        12610.674 ]], dtype=float32)

In [10]:
adata.obsm['mean2_norm']

KeyError: 'mean2_norm'

In [ ]:
adata.obsm['X_enzyme_activity'].min()